In [1]:
%run ../Python_files/util_data_storage_and_load.py

In [2]:
%run ../Python_files/load_dicts.py

In [3]:
%run ../Python_files/util.py

In [4]:
# load logit_route_choice_probability_matrix
P = zload('../temp_files/logit_route_choice_probability_matrix.pkz')

In [5]:
import numpy as np
from numpy.linalg import inv

P = np.matrix(P)

In [6]:
np.size(P, 0), np.size(P, 1)

(56, 140)

In [7]:
import numpy as np
from numpy.linalg import inv

def samp_cov(x):
    """
    x: sample matrix, each column is a link flow vector sample; 24 * K
    K: number of samples
    S: sample covariance matrix
    ----------------
    return: inv(S)
    ----------------
    """
    x = np.matrix(x)
    K = np.size(x, 1)
    x_mean = sum(x[:,k] for k in range(K)) / K
    S = sum(np.dot(x[:,k] - x_mean, np.transpose(x[:,k] - x_mean)) for k in range(K)) / (K - 1)
    return S

In [8]:
# load path-link incidence matrix
A = zload('../temp_files/path-link_incidence_matrix.pkz')

In [9]:
# load link counts data

import json

with open('../temp_files/link_day_minute_Apr_dict_JSON.json', 'r') as json_file:
    link_day_minute_Apr_dict_JSON = json.load(json_file)

In [10]:
week_day_Apr_list = [2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27, 30]

In [11]:
link_day_minute_Apr_list = []
for link_idx in range(24):
    for day in week_day_Apr_list: 
        for minute_idx in range(120):
            key = 'link_' + str(link_idx) + '_' + str(day)
            link_day_minute_Apr_list.append(link_day_minute_Apr_dict_JSON[key] ['NT_flow_minute'][minute_idx])

In [12]:
len(link_day_minute_Apr_list)

60480

In [13]:
x = np.matrix(link_day_minute_Apr_list)
x = np.matrix.reshape(x, 24, 2520)

x = np.nan_to_num(x)
y = np.array(np.transpose(x))
y = y[np.all(y != 0, axis=1)]
x = np.transpose(y)
x = np.matrix(x)

In [14]:
np.size(x, 0), np.size(x, 1)

(24, 2516)

In [15]:
x[:,:2]

matrix([[ 10271.6381037 ,   8141.65487117],
        [  7351.6104606 ,   7786.97735589],
        [ 16557.24538454,  15705.88434318],
        [ 14046.45432394,  15652.60320729],
        [ 12968.08478145,  15968.49104371],
        [ 10044.35017536,  12857.52085255],
        [ 11130.93656799,  12099.20251847],
        [  9235.00698795,  10256.02077644],
        [ 16149.29623624,  15486.53934037],
        [ 17294.60581251,  14244.57034718],
        [ 15831.45532382,  21130.0866284 ],
        [ 13629.66712122,  14670.56595316],
        [ 10566.75246675,  10960.51705714],
        [ 14920.65070766,  15729.51172334],
        [ 10024.82601824,  11449.71437314],
        [  6792.52410589,  11483.79560689],
        [  9844.27865952,  11906.50245041],
        [  7347.41078498,   9262.16862339],
        [ 19873.07433901,  15037.86238591],
        [ 10113.56772777,   9861.76345304],
        [ 11484.60037876,  10115.08064786],
        [  9568.84965789,   8514.81356368],
        [ 10262.94298089,  14613

In [16]:
np.size(A,0), np.size(A,1)

(24, 140)

In [17]:
from gurobipy import *

L = 56  # dimension of lam

K = np.size(x, 1)
S = samp_cov(x)
inv_S = inv(S)

A_t = np.transpose(A)
P_t = np.transpose(P)
# PA'
PA_t = np.dot(P, A_t)
# AP_t
AP_t = np.transpose(PA_t)

Q = np.dot(np.dot(PA_t, inv_S), AP_t)
b = sum([np.dot(np.dot(PA_t, inv_S), x[:, k]) for k in range(K)])


model = Model("OD_matrix_estimation")

lam = []
for l in range(L):
    lam.append(model.addVar(name='lam_' + str(l)))

model.update() 

# Set objective: (K/2) lam' * Q * lam - b' * lam
obj = 0
for i in range(L):
    for j in range(L):
        obj += (1.0 /2) * K * lam[i] * Q[i, j] * lam[j]
for l in range(L):
    obj += - b[l] * lam[l]
model.setObjective(obj)

# Add constraint: lam >= 0
for l in range(L):
    model.addConstr(lam[l] >= 0)

model.update() 

model.optimize()

lam_list = []
for v in model.getVars():
    print('%s %g' % (v.varName, v.x))
    lam_list.append(v.x)
print('Obj: %g' % obj.getValue())

Optimize a model with 56 rows, 56 columns and 56 nonzeros
Model has 1596 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [1e-02, 9e+00]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 56 rows and 0 columns
Presolve time: 0.00s
Presolved: 0 rows, 56 columns, 0 nonzeros
Presolved model has 1596 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 24
 AA' NZ     : 2.760e+02
 Factor NZ  : 3.000e+02
 Factor Ops : 4.900e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -5.18249404e+06  0.00000000e+00  5.76e+03 3.15e+02  8.79e+05     0s
   1   1.37268798e+06 -3.39859849e+06  3.59e+02 5.77e+01  1.41e+05     0s
   2   6.82439168e+04 -1.03720403e+06  3.59e-04 5.68e-14  1.97e+04     0s
   3  -2.28354861e+05 -3.60408931e+05  3.60e-10 5.68e-14  2.3

In [18]:
# write estimation result to file
n = 8  # number of nodes
with open('../temp_files/OD_demand_matrix_Apr_weekday_NT.txt', 'w') as the_file:
    idx = 0
    for i in range(n + 1)[1:]:
        for j in range(n + 1)[1:]:
            if i != j: 
                the_file.write("%d,%d,%f\n" %(i, j, lam_list[idx]))
                idx += 1

In [19]:
lam_list

[1.4349819176517333e-09,
 14343.18039112415,
 1.575137289308894e-09,
 1504.263377307014,
 2892.604066297441,
 0.00031227535471945594,
 4.7460734266718946e-07,
 0.0058732295984781635,
 2373.9227426741486,
 10371.268852506195,
 656.8452925831161,
 1025.988582829719,
 0.00031227378941927437,
 0.0003140621314482112,
 12080.421231538494,
 7321.1654930386685,
 1331.1774298850553,
 2073.1577813857452,
 3684.5458278177116,
 21206.01728730173,
 0.00031236771606116785,
 5304.075233990058,
 5451.903830272259,
 2527.1058518485283,
 3.2030466236105894,
 5827.683192743084,
 0.00017903748140284613,
 8651.340494121623,
 1.3114504882268734,
 1.3079641122463435,
 1743.6488576939548,
 30.905893640739578,
 1412.284053799015,
 0.0001772431642046528,
 0.00017733310327981114,
 2723.7286167970196,
 2014.9135740243657,
 0.0009554475919237703,
 5612.382084851832,
 1384.7840167332406,
 362.7792838883091,
 343.41622545298867,
 2.2573513270370232e-05,
 2.2573511291884335e-05,
 21454.97172883818,
 1385.524681718780